In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import src.config as config

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.4) by running 'pip install hopsworks==3.4.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/413849
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using ArrowFlight (2.09s) 


In [6]:
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
182030,2023-12-29 02:00:00+00:00,0,1
9439,2023-12-29 03:00:00+00:00,0,1
67943,2023-12-29 04:00:00+00:00,0,1
154876,2023-12-29 05:00:00+00:00,0,1
108242,2023-12-29 06:00:00+00:00,1,1
...,...,...,...
129193,2024-01-27 04:00:00+00:00,8,265
122700,2024-01-27 05:00:00+00:00,3,265
129727,2024-01-27 06:00:00+00:00,2,265
130600,2024-01-27 07:00:00+00:00,1,265


In [7]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

In [8]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [00:01<00:00, 240.85it/s]

features_and_target.shape=(530, 675)


In [9]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-26 02:00:00+00:00,1,0.0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2024-01-27 01:00:00+00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-26 02:00:00+00:00,2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-27 01:00:00+00:00,2,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-01-26 02:00:00+00:00,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,49.0,21.0,22.0,17.0,9.0,24.0,30.0,36.0,54.0,72.0,...,195.0,156.0,152.0,153.0,166.0,155.0,128.0,2024-01-27 01:00:00+00:00,263,97.0
526,10.0,6.0,5.0,5.0,16.0,20.0,38.0,50.0,50.0,86.0,...,111.0,90.0,92.0,86.0,50.0,22.0,10.0,2024-01-26 02:00:00+00:00,264,6.0
527,29.0,15.0,18.0,5.0,6.0,12.0,22.0,26.0,38.0,65.0,...,94.0,108.0,80.0,87.0,69.0,76.0,64.0,2024-01-27 01:00:00+00:00,264,41.0
528,4.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0,5.0,4.0,...,0.0,1.0,2.0,4.0,8.0,4.0,4.0,2024-01-26 02:00:00+00:00,265,4.0


In [11]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

features_and_target

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2023-12-30 00:00:00+0000', tz='UTC')
X_train.shape=(0, 674)
y_train.shape=(0,)
X_test.shape=(530, 674)
y_test.shape=(530,)


In [12]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [13]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2024-01-27 14:02:46,173] A new study created in memory with name: no-name-1c748c15-cd4b-4940-8bf0-fc0ab8833e9b
[W 2024-01-27 14:02:46,177] Trial 0 failed with parameters: {'num_leaves': 189, 'feature_fraction': 0.2675561923250389, 'bagging_fraction': 0.2912141472473231, 'min_child_samples': 99} because of the following error: ValueError('Cannot have number of folds=3 greater than the number of samples=0.').
Traceback (most recent call last):
  File "c:\Users\billy.CDT\AppData\Local\pypoetry\Cache\virtualenvs\src-igXaKoE_-py3.11\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\billy.CDT\AppData\Local\Temp\ipykernel_31748\4238820799.py", line 26, in objective
    for train_index, val_index in tss.split(X_train):
  File "c:\Users\billy.CDT\AppData\Local\pypoetry\Cache\virtualenvs\src-igXaKoE_-py3.11\Lib\site-packages\sklearn\model_selection\_split.py", line 1177, in split
    raise

ValueError: Cannot have number of folds=3 greater than the number of samples=0.

In [ ]:
best_params = study.best_trial.params
print(f'{best_params=}')

In [ ]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

In [ ]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')